# AutoTS Example

AutoTS是用于支持PaddleTS的自动机器学习能力组件。

AutoTS 可以支持 PaddleTS 模型和 pipeline 的自动超参数选择，减少人工介入成本，降低专业门槛。

## 1. 准备数据

In [1]:
from paddlets.datasets.repository import get_dataset
tsdataset = get_dataset("UNI_WTH")

/home/work/anaconda3/envs/py37/lib/python3.7/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/home/work/anaconda3/envs/py37/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/work/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/compat/pandas.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  version = LooseVersion(pd.__version__)
/home/work/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/compat/pandas.py:14: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  pandas_lt_1_0_0 = version < LooseVersion('1.0.0')
/home/work/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/compat/pandas.py:15: Depreca

## 2 构造和训练

通过四行代码，我们利用 MLPRegressor 初始化了一个 AutoTS 模型。 AutoTS 会在训练的过程中自动进行超参优化


In [2]:
from paddlets.models.forecasting import MLPRegressor
from paddlets.automl.autots import AutoTS
autots_model = AutoTS(MLPRegressor, 96, 2)
autots_model.fit(tsdataset, n_trials=1)

[2022-11-03 20:02:47,407] [ray.tune.tune] [INFO] Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.
2022-11-03 20:02:51,789	WARNING function_runner.py:604 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
[I 2022-11-03 20:02:51,811] A new study created in memory with name: optuna
(pid=11305) /home/work/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/compat/pandas.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=11305)   version = LooseVersion(pd.__version__)
(pid=11305) /home/work/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/compat/pandas.py:14: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=11305)   pandas_lt_1_0_0 = 

Trial name,status,loc,batch_size,hidden_config,max_epochs,optimizer_params/learning_rate,patience,use_bn
run_trial_71ac71fc,RUNNING,10.182.62.13:11305,56,Choice_3: [128],60,0.00449983,40,False


(run_trial pid=11305) [2022-11-03 20:02:59,608] [paddlets.automl.optimize_runner] [INFO] trial config: {'hidden_config': 'Choice_3: [128]', 'use_bn': False, 'batch_size': 56, 'max_epochs': 60, 'optimizer_params': {'learning_rate': 0.004499832883384026}, 'patience': 40}
(run_trial pid=11305) [2022-11-03 20:02:59,609] [paddlets.automl.optimize_runner] [INFO] setup_estimator: init model. Params: {'hidden_config': [128], 'use_bn': False, 'batch_size': 56, 'max_epochs': 60, 'optimizer_params': {'learning_rate': 0.004499832883384026}, 'patience': 40, 'in_chunk_len': 96, 'out_chunk_len': 2, 'skip_chunk_len': 0, 'sampling_stride': 1, 'seed': 2022}
(run_trial pid=11305) [2022-11-03 20:03:02,153] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 000| loss: 2.856326| val_0_mae: 1.229897| 0:00:02s
(run_trial pid=11305) [2022-11-03 20:03:04,140] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 001| loss: 1.160878| val_0_mae: 0.775531| 0:00:04s


Trial name,status,loc,batch_size,hidden_config,max_epochs,optimizer_params/learning_rate,patience,use_bn
run_trial_71ac71fc,RUNNING,10.182.62.13:11305,56,Choice_3: [128],60,0.00449983,40,False


(run_trial pid=11305) [2022-11-03 20:03:06,285] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 002| loss: 0.998960| val_0_mae: 0.755660| 0:00:06s
(run_trial pid=11305) [2022-11-03 20:03:08,333] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 003| loss: 0.947306| val_0_mae: 0.722077| 0:00:08s


...

Trial name,status,loc,batch_size,hidden_config,max_epochs,optimizer_params/learning_rate,patience,use_bn
run_trial_71ac71fc,RUNNING,10.182.62.13:11305,56,Choice_3: [128],60,0.00449983,40,False


Trial name,status,loc,batch_size,hidden_config,max_epochs,optimizer_params/learning_rate,patience,use_bn
run_trial_71ac71fc,RUNNING,10.182.62.13:11305,56,Choice_3: [128],60,0.00449983,40,False


Result for run_trial_71ac71fc:
  date: 2022-11-03_20-05-36
  done: false
  experiment_id: 4d226194803a4dd89f3e557c762a86ca
  hostname: yq01-m42-xi4-25-for-other-luanwei-863588.yq01.baidu.com
  iterations_since_restore: 1
  loss: 0.6750041246414185
  node_ip: 10.182.62.13
  pid: 11305
  time_since_restore: 157.33996558189392
  time_this_iter_s: 157.33996558189392
  time_total_s: 157.33996558189392
  timestamp: 1667477136
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 71ac71fc
  warmup_time: 0.005208730697631836
  
Result for run_trial_71ac71fc:
  date: 2022-11-03_20-05-36
  done: true
  experiment_id: 4d226194803a4dd89f3e557c762a86ca
  experiment_tag: 1_batch_size=56,hidden_config=Choice_3_128,max_epochs=60,learning_rate=0.0045,patience=40,use_bn=False
  hostname: yq01-m42-xi4-25-for-other-luanwei-863588.yq01.baidu.com
  iterations_since_restore: 1
  loss: 0.6750041246414185
  node_ip: 10.182.62.13
  pid: 11305
  time_since_restore: 157.33996558189392
  time_this_iter_

Trial name,status,loc,batch_size,hidden_config,max_epochs,optimizer_params/learning_rate,patience,use_bn,iter,total time (s),loss
run_trial_71ac71fc,TERMINATED,10.182.62.13:11305,56,Choice_3: [128],60,0.00449983,40,False,1,157.34,0.675004


2022-11-03 20:05:37,090	INFO tune.py:748 -- Total run time: 165.31 seconds (164.51 seconds for the tuning loop).
[2022-11-03 20:05:37,102] [paddlets.automl.autots] [INFO] AutoTS: start refit
[2022-11-03 20:05:37,103] [paddlets.automl.optimize_runner] [INFO] setup_estimator: init model. Params: {'hidden_config': [128], 'use_bn': False, 'batch_size': 56, 'max_epochs': 60, 'optimizer_params': {'learning_rate': 0.004499832883384026}, 'patience': 40, 'in_chunk_len': 96, 'out_chunk_len': 2, 'skip_chunk_len': 0, 'sampling_stride': 1, 'seed': 2022}
[2022-11-03 20:05:39,823] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 000| loss: 2.879014| val_0_mae: 1.180042| 0:00:02s
[2022-11-03 20:05:41,681] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 001| loss: 1.190439| val_0_mae: 0.856322| 0:00:04s
[2022-11-03 20:05:43,801] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 002| loss: 1.037609| val_0_mae: 0.884753| 0:00:06s
[2022-11-03 20:05:45,881] [paddlets.models.commo

## 3. 搜索空间
对于超参数优化来说，你可以定义一个搜索空间。如果你没有指定一个搜索空间，我们也为每个 PaddleTS 模型内置了推荐的默认搜索空间。

你可以利用搜索空间去控制你的超参的取值是如何采样的，控制值范围是多少。

下面是一个指定了搜索空间的 autots pipeline 的例子：

In [3]:
from ray.tune import uniform, qrandint, choice
from paddlets.transform import Fill

sp = {
    "Fill": {
        "cols": ['WetBulbCelsius'],
        "method": choice(['max', 'min', 'mean', 'median', 'pre', 'next', 'zero']),
        "value": uniform(0.1, 0.9),
        "window_size": qrandint(20, 50, q=1)
    },
    "MLPRegressor": {
        "batch_size": qrandint(16, 64, q=16),
        "use_bn": choice([True, False]),
        "max_epochs": qrandint(10, 50, q=10)
    }
}
autots_model = AutoTS([Fill, MLPRegressor], 25, 2, search_space=sp, sampling_stride=25)
autots_model.fit(tsdataset, n_trials=1)
sp = autots_model.search_space()
predicted = autots_model.predict(tsdataset)

[I 2022-11-03 20:07:40,140] A new study created in memory with name: optuna
(pid=26976) /home/work/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/compat/pandas.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=26976)   version = LooseVersion(pd.__version__)
(pid=26976) /home/work/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/compat/pandas.py:14: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=26976)   pandas_lt_1_0_0 = version < LooseVersion('1.0.0')
(pid=26976) /home/work/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/compat/pandas.py:15: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(pid=26976)   pandas_lt_25_0 = version < LooseVersion('0.25.0')
(pid=26976) /home/work/anaconda3/envs/py37/lib/python3.7/site-packages/statsmodels/compat/pandas.py:16: DeprecationWarning: distutils Version classes ar

Trial name,status,loc,Fill/method,Fill/value,Fill/window_size,MLPRegressor/batch_size,MLPRegressor/max_epochs,MLPRegressor/use_bn
run_trial_1d1fb81e,RUNNING,10.182.62.13:26976,zero,0.589486,29,32,50,False


(run_trial pid=26976) [2022-11-03 20:07:47,328] [paddlets.automl.optimize_runner] [INFO] trial config: {'Fill': {'cols': ['WetBulbCelsius'], 'method': 'zero', 'value': 0.5894864023664573, 'window_size': 29}, 'MLPRegressor': {'batch_size': 32, 'use_bn': False, 'max_epochs': 50}}
(run_trial pid=26976) [2022-11-03 20:07:47,330] [paddlets.automl.optimize_runner] [INFO] setup_estimator: init pipeline. Params: [(<class 'paddlets.transform.fill.Fill'>, {'cols': ['WetBulbCelsius'], 'method': 'zero', 'value': 0.5894864023664573, 'window_size': 29}), (<class 'paddlets.models.forecasting.dl.mlp.MLPRegressor'>, {'batch_size': 32, 'use_bn': False, 'max_epochs': 50, 'in_chunk_len': 25, 'out_chunk_len': 2, 'skip_chunk_len': 0, 'sampling_stride': 25, 'seed': 2022})]
(run_trial pid=26976) [2022-11-03 20:07:47,599] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 000| loss: 7.908529| val_0_mae: 1.541964| 0:00:00s
(run_trial pid=26976) [2022-11-03 20:07:47,740] [paddlets.models.common.callbacks.

Trial name,status,loc,Fill/method,Fill/value,Fill/window_size,MLPRegressor/batch_size,MLPRegressor/max_epochs,MLPRegressor/use_bn
run_trial_1d1fb81e,RUNNING,10.182.62.13:26976,zero,0.589486,29,32,50,False


(run_trial pid=26976) [2022-11-03 20:07:52,376] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 036| loss: 1.207132| val_0_mae: 0.800414| 0:00:04s
(run_trial pid=26976) [2022-11-03 20:07:52,539] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 037| loss: 1.102531| val_0_mae: 0.807719| 0:00:05s
(run_trial pid=26976) [2022-11-03 20:07:52,673] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 038| loss: 1.143725| val_0_mae: 0.774669| 0:00:05s
(run_trial pid=26976) [2022-11-03 20:07:52,789] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 039| loss: 1.090562| val_0_mae: 0.760797| 0:00:05s
(run_trial pid=26976) [2022-11-03 20:07:52,904] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 040| loss: 1.129319| val_0_mae: 0.790541| 0:00:05s
(run_trial pid=26976) [2022-11-03 20:07:53,019] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 041| loss: 1.178506| val_0_mae: 0.761641| 0:00:05s
(run_trial pid=26976) [2022-11-03 20:07:53,136] [paddlets.

...

Trial name,status,loc,Fill/method,Fill/value,Fill/window_size,MLPRegressor/batch_size,MLPRegressor/max_epochs,MLPRegressor/use_bn
run_trial_1d1fb81e,RUNNING,10.182.62.13:26976,zero,0.589486,29,32,50,False


Trial name,status,loc,Fill/method,Fill/value,Fill/window_size,MLPRegressor/batch_size,MLPRegressor/max_epochs,MLPRegressor/use_bn
run_trial_1d1fb81e,RUNNING,10.182.62.13:26976,zero,0.589486,29,32,50,False


Result for run_trial_1d1fb81e:
  date: 2022-11-03_20-08-31
  done: false
  experiment_id: 8c851e3e7d054c118baacc3a6fd866c4
  hostname: yq01-m42-xi4-25-for-other-luanwei-863588.yq01.baidu.com
  iterations_since_restore: 1
  loss: 0.7847563280723457
  node_ip: 10.182.62.13
  pid: 26976
  time_since_restore: 44.57377243041992
  time_this_iter_s: 44.57377243041992
  time_total_s: 44.57377243041992
  timestamp: 1667477311
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 1d1fb81e
  warmup_time: 0.0057294368743896484
  
Result for run_trial_1d1fb81e:
  date: 2022-11-03_20-08-31
  done: true
  experiment_id: 8c851e3e7d054c118baacc3a6fd866c4
  experiment_tag: 1_cols=WetBulbCelsius,method=zero,value=0.5895,window_size=29,batch_size=32,max_epochs=50,use_bn=False
  hostname: yq01-m42-xi4-25-for-other-luanwei-863588.yq01.baidu.com
  iterations_since_restore: 1
  loss: 0.7847563280723457
  node_ip: 10.182.62.13
  pid: 26976
  time_since_restore: 44.57377243041992
  time_this_iter_s: 

Trial name,status,loc,Fill/method,Fill/value,Fill/window_size,MLPRegressor/batch_size,MLPRegressor/max_epochs,MLPRegressor/use_bn,iter,total time (s),loss
run_trial_1d1fb81e,TERMINATED,10.182.62.13:26976,zero,0.589486,29,32,50,False,1,44.5738,0.784756


2022-11-03 20:08:32,057	INFO tune.py:748 -- Total run time: 51.94 seconds (51.79 seconds for the tuning loop).
[2022-11-03 20:08:32,071] [paddlets.automl.autots] [INFO] AutoTS: start refit
[2022-11-03 20:08:32,073] [paddlets.automl.optimize_runner] [INFO] setup_estimator: init pipeline. Params: [(<class 'paddlets.transform.fill.Fill'>, {'cols': ['WetBulbCelsius'], 'method': 'zero', 'value': 0.5894864023664573, 'window_size': 29}), (<class 'paddlets.models.forecasting.dl.mlp.MLPRegressor'>, {'batch_size': 32, 'use_bn': False, 'max_epochs': 50, 'in_chunk_len': 25, 'out_chunk_len': 2, 'skip_chunk_len': 0, 'sampling_stride': 25, 'seed': 2022})]
[2022-11-03 20:08:32,242] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 000| loss: 8.221830| val_0_mae: 1.561473| 0:00:00s
[2022-11-03 20:08:32,354] [paddlets.models.common.callbacks.callbacks] [INFO] epoch 001| loss: 3.027953| val_0_mae: 1.395608| 0:00:00s
[2022-11-03 20:08:32,467] [paddlets.models.common.callbacks.callbacks] [INFO] epo

搜索空间定义的 API 可以参考：https://docs.ray.io/en/latest/tune/api_docs/search_space.html

## 4. 搜索算法

PaddleTS 的搜索算法是利用多个开源的超参优化库包装而成。

我们内置了如下几种搜索算法：
[“Random”, “CMAES”, “TPE”, “CFO”, “BlendSearch”, “Bayes”]

对于这些超参优化库的细节，可以参考它们对应的开源文档。

你可以通过下面的方式指定搜索算法：



In [4]:
autots_model = AutoTS(MLPRegressor, 96, 2, search_space="CMAES")

如果搜索算法没有被指定，我们会默认使用TPE算法。